# DAX and Measures

> *DAX (Data Analysis Expressions)* is a powerful formula language used in Power BI's Model View and Report View to derive new features and insights from the existing data in your model. It employs a collection of functions, operators, and constants that can be combined to build formulas for defining measures, calculated columns and calculated tables.



## DAX Syntax

### Basic Syntactic Structure

A DAX expression starts with an equal sign `=` and is followed by functions, operators, constants, or references to other elements. Here's a simple breakdown:

- **Equal Sign (`=`):** Every DAX formula begins with an equal sign.
- **Function:** Most DAX expressions one of the array of 200+ functions in the DAX library, for example `SUM()`, `AVERAGE()`, or `COUNT()`
- **Operators:** DAX uses operators to perform arithmetic, comparison, and text operations
- **Constants:** DAX expressions can also contain static values, e.g. numbers like `5` or text strings enclosed in double quotes like `"Product"`

### Referencing in DAX

- **Tables:** Just use the table name to reference a table. For example: `Orders`
- **Columns:** Reference a column by enclosing the table name and column name in square brackets. For instance: `[Order Date]` or `Orders[Order Date]`
- **Measures:** When referencing measures, use the measure name directly or with the table name. Example: `[Total Sales]` or `Orders[Total Sales]`

### Basic Operators

DAX includes a variety of operators, some of the most common ones are:

- **Arithmetic Operators:** `+`, `-`, `*`, `/`
- **Comparison Operators:** `=`, `>`, `<`, `>=`, `<=`, `<>` (not equal to)
- **Text Concatenation:** Use the `&` operator to combine two text strings.
- **Logical Operators:** `&&` (AND), `||` (OR), `NOT`

### Additional Tips

1. **Nesting Functions:** DAX expressions can be nested, meaning you can use the result of one function as the argument of another, e.g., `SUMX(FILTER(Orders, Orders[Unit Price] > 100), Orders[Number of Items])`.
2. **Context Awareness:** Always remember that DAX formulas are context-aware. They'll change their result based on the filter and row context.
3. **Comments:** In DAX, you can include comments in your code using `//` for single-line comments or `/* */` for multi-line comments. This is useful for documentation and understanding complex formulas later.


## Measures in DAX

In Power BI, a **Measure** is a calculation that’s performed on data in the data model. The calculation is the result of evaluating a DAX expression, and to be a measure, this must evaluate to a single value, as opposed to a calculated column, which will evaluate to a single value per row in the source table. Measures are used to calculate results "on the fly" as the user explores and interacts with visuals on Power BI reports. This means that a measure will obey the filter context in which it is used.

To understand this, let's look at a simple example. Say we hade an order table, whose head 5 rows look like this:

| Order ID | Order Date | Number of Items | Unit Price | Product ID |
|----------|------------|-----------------|------------|------------|
| 1001     | 01/01/2023 | 2               | $100       | P001       |
| 1002     | 01/02/2023 | 1               | $50        | P002       |
| 1003     | 01/03/2023 | 3               | $100       | P001       |
| 1004     | 01/04/2023 | 4               | $40        | P004       |
| 1005     | 01/05/2023 | 1               | $250       | P005       |

We could define a simple DAX measure for the total number of orders, as follows:

- Navigate to the Power BI Fields list and select the appropriate table.
- Right-click and choose **New measure**.
- In the formula bar, write the DAX formula:

```DAX
Total Orders = COUNT(Orders[Order ID])
```
<div style="text-align: center;">
<img src="images/total_orders.gif" width="1200"> <br> <br>
</div>
The DAX expression here simply returns the count of the elements of the `[Order ID]` column in the `Orders` table, and in its base form would evaluate to the number of rows in the table. However, as measures obey filter context, this same measure could be used as the Y-axis value in a bar chart visual showing the count of orders for each Product ID. For each bar in the chart, the filter context would be the specific product ID for that bar.

<div style="text-align: center;">
<img src="images/total_orders_chart.png" width="700"> <br> <br>
</div>

## Calculated Columns in DAX

A **Calculated Column** is a custom column added to a table in your data model. For every row in the table, the calculated column has a value that is the result of a DAX formula. The formula is evaluated once for each row, considering the row context.

### Let's Dive In With an Example:

Using the order table provided:

| Order ID | Order Date | Number of Items | Unit Price | Product ID |
|----------|------------|-----------------|------------|------------|
| 1001     | 01/01/2023 | 2               | $100       | P001       |
| ...      | ...        | ...             | ...        | ...       |

If we wanted to determine the total price for each order, we could create a calculated column named `[Total Price]`. Here's how:

- Navigate to the Power BI Fields list and choose the `Orders` table.
- Right-click and select **New column**.
- In the formula bar, draft the DAX formula:

```DAX
Total Price = Orders[Number of Items] * Orders[Unit Price]
```
<div style="text-align: center;">
<img src="images/total_price.gif" width="1000"> <br> <br>
</div>
The DAX formula multiplies the number of items by the unit price for each row in the `Orders` table, generating a new column with the total price for every order.

## DAX functions

## Aggregation Functions in DAX

In DAX, Aggregation Functions help summarize and analyze data by performing calculations on a group of specific data points, returning a single value. They include summation, counting, and returning average values, percentiles and min/max values.

### Primary Aggregation Functions:

- **SUM:** Computes the total of a column's values.
  
  ```DAX
  Total Sales = SUM(Sales[Revenue])
  ```
  
- **AVERAGE:** Determines the mean value of a column.

  ```DAX
  Average Revenue = AVERAGE(Sales[Revenue])
  ```
  
- **COUNT:** Enumerates the number of rows containing data in a column.

  ```DAX
  Order Count = COUNT(Orders[Order ID])
  ```
  
- **MAX:** Identifies the highest value within a column.

  ```DAX
  Maximum Sale = MAX(Sales[Revenue])
  ```
  
- **MIN:** Pinpoints the lowest value inside a column.

  ```DAX
  Minimum Sale = MIN(Sales[Revenue])
  ```


### Extended Aggregation Functions: The “X” Factor

Now, while functions like `SUM` and `COUNT` operate directly on columns, their "X" counterparts, like `SUMX` and `COUNTX`, offer more versatility. They allow you to iterate over a table or a table expression and then perform aggregations.

They have the general formula:

```DAX
 X_FUNCTION(Table,expression)
```

- **SUMX:** Iterates over a table and sums up the result of an expression for each row.

For example, imagine you have a sales database where each product sold might have a unique discount applied. While the `Sales` table lists each product's base price and the discount percentage, the final sale price after applying the discount isn't directly available.


| Product Name | Base Price | Discount Percentage |
|--------------|------------|----------------------|
| Laptop      | $1000      | 10%                  |
| Phone       | $500       | 5%                   |
| Headphones  | $100       | 2%                   |

You want to create a measure to calculate the total revenue after applying the discounts to each product. Instead of creating a separate calculated column for the discounted price and then summing that column, you can use `SUMX` to iterate over the `Sales` table, calculate the discounted price for each product, and then sum these values together.

The DAX formula would look like this:

```DAX
Total Discounted Sales = SUMX(Sales, Sales[Base Price] * (1 - Sales[Discount Percentage]))
```

In this formula, for each row in the `Sales` table, `SUMX` calculates the price after discount and then aggregates (sums up) all these calculated values to give the total discounted sales.


- **COUNTX:** Counts the number of rows in a table, dependent on the evaluation of an expression


For example, if you have a product sales table and you're interested in identifying how many products have achieved sales above a certain threshold. Let's say you want to know how many products have sold more than 100 units.



| Product Name | Units Sold |
|--------------|------------|
| Laptop      | 150        |
| Phone       | 90         |
| Headphones  | 110        |
| Keyboard    | 50         |
| Mouse       | 105        |

Instead of creating a separate calculated column that flags products with sales over 100 units and then counting those flags, you can use `COUNTX` to iterate over the `Product Sales` table and directly count the products meeting the criterion.

The DAX formula would look like this:

```DAX
Products Over 100 Sales = COUNTX(Product Sales, IF(Product Sales[Units Sold] > 100, 1))
```

Here, `COUNTX` iterates over each row in the `Product Sales` table. For each product with units sold over 100, the IF function returns 1. `COUNTX` then counts these 1's, giving the total number of products with significant sales.


### Differences Between Standard and Extended Forms:

1. **Direct vs. Iterative:** Standard aggregation functions like `SUM` and `AVERAGE` work directly on entire columns, but their extended counterparts iterate through rows, performing calculations based on expressions
2. **Flexibility:** "X" functions like `SUMX` or `COUNTX` offer more complex aggregations, enabling custom operations on each row of a table before the aggregation
3. **Performance Consideration:** Iterative functions, by nature, might be slower, especially with large datasets, because they evaluate the expression row-by-row


## Filtering in Power BI

In Power BI, **filtering** lets you focus on specific data segments. Instead of viewing every row of data, you can zoom in on specific subsets, like transactions in a specific month or sales of a single product. Filtering refines data based on defined conditions or criteria.

### Filter Context

When you construct a report in Power BI, every visual works with DAX under the hood. When you craft or adjust a visual, it establishes a filter context. This context can be implicit, derived from the visual's construction, or explicit, from applying filter conditions manually.

Filter context is the environment in which a DAX formula is evaluated. When you introduce a measure to a report, its values adjust based on the context. For example, if you plot a measure like `Total Revenue` on a bar chart segmented by product category, the product category is an implicit or automatic filter context, which causes the measure to evaluate differently on each bar. However,you might add an additional filter to restrict the filter context of the entire visual to a specific year, so that only that year's data are displayed on the chart.

Every visual in Power BI has an underlying filter context influenced by several elements, from slicers to other visuals. As a result, a measure's value can shift based on its placement or interaction within a report.

We will delve deeper into the specifics of filtering and filter context in reports in another lesson. For now we will consider filters at the DAX level specifically.

[PLACEHOLDER: filtering example]

## DAX Filter Functions

Some of the most important and widely-used DAX functions either apply or modify filter context. It is these functions that are being called under the hood when visuals are constructed, but they are also used directly in the creation of specific measures and columns.

### `FILTER`

The `FILTER` function creates a new table that includes only the rows that satisfy a certain condition. You can use `FILTER` to reduce the number of rows in the table that you are working with, and use only specific data in calculations. `FILTER` is not used independently, but as a function that is embedded in other functions that require a table as an argument.

```
CALCULATE(SUM(Sales[SalesAmount]), FILTER(Sales, Sales[Year]=2020))
```

The `FILTER` function in the above example creates a new table that includes data only from the year 2020, which is then passed to the `CALCULATE` function, which sums `SalesAmount` for that data.

### The `CALCULATE` Function

The `CALCULATE` function evaluates an expression in a context modified by filters. It's syntax is `CALCULATE(<expression>,<filter1>,<filter2>,...)`. A good way to think about this is that when you see `CALCULATE`, you should think "Ignore current filter context". In effect, `CALCULATE` overrides whatever the current filter context is, and replaces it with the context specified by its own filter arguments.


```dax
Revenue East = CALCULATE(SUM(Sales[SalesAmount]), Sales[Region] = "East")
```

In above example, the `CALCULATE` function calculates the total SalesAmount for just the East Region, and assigns it to the measure `Revenue East`. This measure would always return the revenue for the East region, even if it was used in a visual with a different context. So for example, in a bar chart broken down by product category, it would apply the product category filter, but would only show values for the `EAST` region. But in a bar chart broken down by region, it would show the correct value for the `EAST` region, but all other regions would show a value of zero, as both the visual's region filter, and the filter inside the `CALCULATE` function would be applied.

[animated gif: steps to show the use of CALCULATE function]

The `CALCULATE` function can use multiple filters:

```dax
CALCULATE(SUM(Sales[SalesAmount]), Sales[Region] = "East", Sales[Year] = 2020)
```

With multiple filters, the `CALCULATE` function in the above example computes the total SalesAmount for region "East" in year 2020.


#### CALCULATE Modifiers

The `CALCULATE` function includes modifiers that can further refine the filtering - `ALL`, `ALLEXCEPT`, `ALLSELECTED`, `VALUES`, `RELATED` and `USERELATIONSHIP`. We will cover the most important of these in detail below.




### `USERELATIONSHIP`

`USERELATIONSHIP` function is used in `CALCULATE` to specify a relationship to be used in a calculation. An example of where this is useful is in situations where you have multiple date fields in a table (eg. `[Order Date]` and `[Ship Date]`), connected to a single date table. Power BI can only have one active relationship between tables at a time, so if you have multiple relationships between the same pair of table, one will be active by default, but you can use the others in a measure with the `USERELATIONSHIP` function.

For example, the table schema below is an example of a situation where there are two relationships between a pair of tables. The relationship between  `Orders[Order Date]`  and `Date[Date]` is active by default:


<div style="text-align: center;">
    <img src="images/double_rel.png" width="600">
</div>


If we wanted to calculate total sales based on the date the orders were shipped, rather than the date they were placed, we could use the following DAX expression to create the measure:

```DAX
Total Sales by Ship Date :=
CALCULATE(
    SUM(Orders[Amount]),
    USERELATIONSHIP(Orders[ShipDate], Date[Date])
)
```



### The `ALL` Function

The `ALL` function removes all context filters in the table it is being used on, essentially showing all the data in the table.

```dax
CALCULATE(SUM(Sales[SalesAmount]), ALL(Sales))
```

In the above example, the `ALL` function removes all filters applied on the Sales table, and the `CALCULATE` function then sums the SalesAmount for all data in the table.



### Worked Example

Imagine you have a sales database that tracks the sales of different products across various regions. You want to calculate the market share of a specific product in a particular region, for use in a stacked bar chart showing market share of the different products, filtered to the sales region of interest.

<div style="text-align: center;">
<img src="images/market_share.png" width="600"> <br> <br>
</div>

The **Market Share** is typically calculated as:
<br><br>
$$
\text{Market Share} = \frac{\text{Sales of the Product in the Region}}{\text{Total Sales in the Region}}
$$

<br><br>

Let's represent the sales of a particular product with a general total sales measure as follows:
```dax
Product Sales = SUM(Sales[SalesAmount])
```
This gives us a measure that will 

Now, to get the denominator of our market share formula (i.e., Total Sales in the Region), you need to remove the product-level filter to consider all products but retain the region filter. This is where the `ALL` function comes in handy.

You might use `ALL` like this:
```dax
Total Sales = CALCULATE(
                        SUM(Sales[SalesAmount]),
                        ALL(Sales[Product])   // Remove filter from Product column but keep the region context
                    )
```

With these two measures, the market share for a product in a region can be computed as:
```dax
Market Share = DIVIDE(ProductSales, TotalSales)
```

In this scenario, the `ALL` function is crucial to remove the product filter to consider sales of all products in the region while calculating the market share.


## The `RELATED` Function

The `RELATED` function in DAX retrieves a value from a related table using an existing relationship. It is commonly used in scenarios where data from two tables needs to be combined based on a shared key.



Let's look at an example to highlight the utility of the `RELATED` function:

### Scenario:

Consider a company that has a table of sales transactions (`Sales`) and a separate table containing employee details (`Employees`). The goal is to calculate the commission for each sale based on the employee's commission rate.

### Tables:

1. **Sales**:

<div style="text-align: center;">
<img src="images/sales_table.png" width="600"> <br> <br>
</div>

2. **Employees**:

<div style="text-align: center;">
<img src="images/employees_table.png" width="600"> <br> <br>
</div>

### Relationship:

There's a relationship between the `Sales` table and the `Employees` table using the `EmployeeID` column.

### Task:

We need to create a new calculated column in the `Sales` table that shows the commission earned for each transaction. To do this we can use the following DAX formula:


```dax
Commission = Sales[SaleAmount] * RELATED(Employees[CommissionRate])
```

And the result will be as follows:

<div style="text-align: center;">
<img src="images/commissionrate.png" width="600"> <br> <br>
</div>

In this example, while a relationship is present between `Sales` and `Employees`, the `RELATED` function is essential to perform calculations using data from both tables. The function fetches the commission rate for each employee and multiplies it with the sale amount to compute the commission.
